In [ ]:
#paired test for 3 or more scale variables
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
x,y=load_iris(return_X_y=True, as_frame=True)
x.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [ ]:
y.unique()

array([0, 1, 2])

In [ ]:
data=pd.concat([x,y], axis=1)
data

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [ ]:
data['target'].replace({0:'type_1', 1:'type_2', 2:'type_3'})
data.columns

Index(['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)',
       'petal width (cm)', 'target'],
      dtype='object')

In [ ]:
#multivariate analysis of variance

#H0 - the scale variables are the same for the given target/flower_name.
#Ha - the scale variables are not the same for the given target/flower_name.
from statsmodels.multivariate.manova import MANOVA
data=data.rename(columns={
    'sepal length (cm)':'sepal_length',
    'sepal width (cm)':'sepal_width',
    'petal length (cm)':'petal_length',
    'petal width (cm)':'petal_width',
    'target':'flower_name'
})
data.columns

Index(['sepal_length', 'sepal_width', 'petal_length', 'petal width',
       'flower_name'],
      dtype='object')

In [ ]:
res=MANOVA.from_formula('sepal_length + sepal_width + petal_length + petal_width ~ flower_name',data=data)


In [ ]:
print(res.mv_test())

                   Multivariate linear model
                                                                
----------------------------------------------------------------
       Intercept         Value  Num DF  Den DF   F Value  Pr > F
----------------------------------------------------------------
          Wilks' lambda  0.0190 4.0000 145.0000 1875.7577 0.0000
         Pillai's trace  0.9810 4.0000 145.0000 1875.7577 0.0000
 Hotelling-Lawley trace 51.7450 4.0000 145.0000 1875.7577 0.0000
    Roy's greatest root 51.7450 4.0000 145.0000 1875.7577 0.0000
----------------------------------------------------------------
                                                                
----------------------------------------------------------------
       flower_name        Value  Num DF  Den DF  F Value  Pr > F
----------------------------------------------------------------
           Wilks' lambda  0.0696 4.0000 145.0000 484.5379 0.0000
          Pillai's trace  0.9304 4.0000 145.0

In [ ]:
Wilks_lambda = 0.0696 #effect size
pvalue=0.0000
if Wilks_lambda>0.9:
  print('small')
elif Wilks_lambda>0.7 and Wilks_lambda<0.9:
  print('medium')
else:
  print('large')

large


The MANOVA test found that sepal_length + sepal_width + petal_length + petal_width are not same for the different flower name (p<0.05) and further wilks' lambda effect size suggest the difference is large (lambda=0.0696).

In [ ]:
#paired test on 3 or more ordinal variables.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
data=pd.read_csv('StudentStatistics.csv', sep=';')

In [ ]:
selected_data=data[['Teach_Motivate','Teach_LinkTheory','Teach_StimAsk','Teach_Avail']]
selected_data

In [ ]:
selected_data['Teach_Motivate'].unique()

array(['Fully Disagree', 'Disagree', 'Fully agree',
       'Neither disagree nor agree', nan, 'Agree'], dtype=object)

In [ ]:
coding={
    'Fully Disagree':0,
    'Disagree':1,
    'Neither disagree nor agree':2,
    'Agree':3,
    'Fully agree':4
}
selected_data.dropna(inplace=True)
selected_data.replace(coding, inplace=True)
selected_data

In [ ]:
#friedman chi-sqaure test --extention of wilcoxon signed ranked test.
#H0 - the different ordinal variables have the same responses.
#Ha - the different ordinal variables don't have the same responses.
from scipy.stats import friedmanchisquare
friedmanchisquare(selected_data['Teach_Motivate'], selected_data['Teach_LinkTheory'],
                               selected_data['Teach_StimAsk'], selected_data['Teach_Avail'])


FriedmanchisquareResult(statistic=33.85869565217378, pvalue=2.1220432262951938e-07)

In [ ]:
statistic=33.85869565217378
pvalue=2.1220432262951938e-07

#effect size - kendall W

n=selected_data.shape[0]
k=selected_data.shape[1]
W=statistic/(n*(k-1))
W


0.21294777139731935

In [ ]:
if 0<W<0.3:
  print('weak')
elif 0.3<W<0.5:
  print('moderate')
elif 0.5<W<0.7:
  print('relatively strong')
else:
  print('strong')

weak


The different variables don't have the same responses and after applying the friedman test the difference was significant (p<0.05) and further the kendall W effect size siggests that the difference is weak (W=0.21).

In [ ]:
#paired test on 3 or more binary variables.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
data=pd.read_csv('FilmPreferences2123.csv')

In [ ]:
selected_data=data[['munt','movies','tuschin','arena']]

In [ ]:
#cochrans q test
#H0 - four variables are equally selected.
#Ha - four variables are not equally selected.
from statsmodels.stats.contingency_tables import cochrans_q
print(cochrans_q(selected_data))

df          3
pvalue      0.00022763307854354147
statistic   19.384615384615383


In [ ]:
#effect size - eta squared
df    =     3
pvalue  =    0.00022763307854354147
statistic =  19.384615384615383
eta=statistic/(df*selected_data.shape[0])
eta

0.043076923076923075

In [ ]:
if eta<=0.01:
  print('small')
elif eta<=0.06:
  print('moderate')
else:
  print('large')

moderate


on applying the cochrans q test it was found that the four variables is not equally selected (pvalue<0.05) and difference in the selection is moderate (eta=0.04).